# Tutorial 5: Langchain Agent

In this tutorial we explore how to combine Langchain's modular interfaces with a robust C++ runtime, that is portable to mobile also.

## Initial Setup

Before proceeding, please follow the
[instructions](.../tutorial_1_simple_cascade.ipynb) in Tutorial 1 to set up your environment, connect Jupyter, and run the command below to run
the GenC imports we're goign to use.

In [ ]:
import generative_computing.python as genc
from generative_computing.python import authoring
from generative_computing.python import interop
from generative_computing.python import runtime
from generative_computing.python.examples import executor

In [ ]:
import textwrap
import langchain
from langchain.prompts import PromptTemplate

## Define Tools
WolframAlpha is a Tool, therefore we wrap it under a LangChain Tool interface.*italicized text*

In [ ]:
class WolframAlpha(genc.interop.langchain.CustomTool):
  """A specific implementation of CustomTool, including an appid for identification."""

  def __init__(self, appid: str):
    name = "WolframAlpha"
    description = (
        "https://products.wolframalpha.com/api/documentation, one capability is"
        " to evaluate math expression."
    )
    # Extract math equation from a text, then calls WolfraAlpha
    computation = (
        genc.interop.langchain.CustomChain()
        | genc.authoring.create_custom_function("/react/extract_math_question")
        | genc.authoring.create_wolfram_alpha(appid)
        # Template Engine will extract the result from response JSON
        | genc.authoring.create_inja_template(
            "{% if queryresult.pods"
            " %}{{queryresult.pods.0.subpods.0.plaintext}}{% endif %}"
        )
    )

    super().__init__(
        name=name, description=description, computation=computation
    )

## Define Agent with Tool

In this example, we already moved the reusable agent/reasoning code into a LangChain interopertor, so you no longer need to write that.

In this section we explore how to take the prepackaged Agent to speed up your dev process.

In [ ]:
class MathToolAgent(genc.interop.langchain.CustomAgent):
  """An agent that combines ReAct reasoning loop with WolframAlpha tools."""

  def __init__(self, appid: str, api_key: str):

    system_instruction = """
    Solve a question answering task with interleaving Thought, Action,
    Observation steps.
    Thought can reason about the current situation
    Action can be only two types:
    (1) Math[query], Useful for when you need to solve math problems.
    (2) Finish[answer], which returns the answer as a number terminates.
    Here's an example:
    Question: what is the result of power of 2 + 1?
    Thought: power of 2
    Action: Math[pow(2)]
    Observation: 4
    Thought: I can now process the answer.
    Action: Math[4+1]
    Observation: 5
    Thought: Seems like I got the answer.
    Action: Finish[5]
    Please do it step by step.
    Question: {question}
    """

    tools = [WolframAlpha(appid)]

    llm = genc.interop.langchain.CustomModel(
        uri="/cloud/gemini",
        config=genc.interop.gemini.create_config(api_key),
    )

    # Init langchain.agents.agent.Agent with components backed by C++ runtime.
    super().__init__(
        llm_chain=langchain.chains.LLMChain(llm=llm, prompt=PromptTemplate(
          input_variables=["query"],
          template=system_instruction)),
        allowed_tools=[tool.name for tool in tools],
        tools_list=tools,
        max_iterations=8,
    )

## Compile & Run it

In [ ]:
wolfram_app_id=""
gemini_api_key=""

agent = MathToolAgent(wolfram_app_id, gemini_api_key)

portable_ir = genc.interop.langchain.create_computation(agent)

# To run it localy
runner = genc.runtime.Runner(portable_ir,
                             genc.examples.executor.create_default_executor())
result = runner("""
  what is the result of (square root of 4) + 3 to the power of 2 + 3 *(8 +
  4) / 2 - 7"""
)

print(result)

## Save the IR to a file and deploy on Android

IR stands for intermediate representation. GenC transforms the agent into a computation schema or proto, which is platform agnostic. Therefore we can directly load it on a platform with GenC runtime.

1. Run following code to save the IR to a file.

2. Follow instructions [here](.../tutorial_1_simple_cascadeipynb#scrollTo=GhLV1BlXt0fK) to deploy the IR on Android phone.

In [ ]:
# Let's take a peak of how the IR or computation proto looks like
print(portable_ir)

with open("/tmp/genc_demo.pb", "wb") as f:
  f.write(portable_ir.SerializeToString())


## Run the demo on Android

Open the “Generative Computing Demo” app and type a math query.
Sample query to enter in UI:

```What is the result of 6 * 9 + (square root of 4)```

See the result.